In [1]:
import sys
sys.path.append("/persist/maplab_ws/src/ninebot_gio_coupling")
from gnsspylib.gps_time import GPSTime, gpst_to_utc, utc_to_gpst, adjweek, gen_gpstime_from_bds
from gnsspylib.sat_no_prn_id import *
from gnsspylib.gen_nav_msg import gen_nav_bag_msg
from gnsspylib.gen_obs_msg import gen_bag_msg_list
from ninebot_gio_coupling.msg import StampedFloat64Array

In [2]:
import georinex as gr
base_dat = gr.load("/persist/catkin_ws/src/GraphGNSSLib/global_fusion/dataset/gps_solution_TST/hksc1180.19o")
obs_dat = gr.load("/persist/catkin_ws/src/GraphGNSSLib/global_fusion/dataset/gps_solution_TST/COM3_190428_124409.obs")
nav_dat_n = gr.load("/persist/catkin_ws/src/GraphGNSSLib/global_fusion/dataset/gps_solution_TST/hksc1180.19n")
nav_dat_b = gr.load("/persist/catkin_ws/src/GraphGNSSLib/global_fusion/dataset/gps_solution_TST/hksc1180.19b")

In [3]:
obs_bag_msg_list = gen_bag_msg_list(obs_dat, '/gnss/meas')
print(obs_bag_msg_list[0][0], obs_bag_msg_list[-1][0])
base_timestamp_lambda = lambda x : x > obs_bag_msg_list[0][0] - 60 and x < obs_bag_msg_list[-1][0] + 60
# base_timestamp_lambda(1560089800)
base_bag_msg_list = gen_bag_msg_list(base_dat, '/gnss/base/meas', timestamp_lambda=base_timestamp_lambda)

1556455473.996999 1556457233.000999


In [4]:
ephem_timestamp_lambda = lambda x : x > obs_bag_msg_list[0][0] - 7300 and x < obs_bag_msg_list[-1][0] + 7300
b_nav_msg_list = gen_nav_bag_msg(nav_dat_b, timestamp_lambda=ephem_timestamp_lambda)
n_nav_msg_list = gen_nav_bag_msg(nav_dat_n, timestamp_lambda=ephem_timestamp_lambda)
bag_msg_list = obs_bag_msg_list + base_bag_msg_list
bag_msg_list.sort(key=lambda x: x[0])
print("{} -> {}".format(bag_msg_list[0][0], bag_msg_list[-1][0]))

In [6]:
print(len(b_nav_msg_list), len(n_nav_msg_list))

69 27


In [7]:
all_nav_msg_list_tmp = b_nav_msg_list + n_nav_msg_list
all_nav_msg_list = []
i = 0
for timestamp, bag_msg, topic_name in all_nav_msg_list_tmp:
    # new_timestamp = timestamp - 7200
    # if new_timestamp < bag_msg_list[0][0] - 5:
    #     new_timestamp = bag_msg_list[0][0] - 5
    # if new_timestamp > bag_msg_list[-1][0] + 5:
    #     continue
    all_nav_msg_list.append((bag_msg_list[0][0] - 6 + 0.01 * i, bag_msg, topic_name))
    i = i + 1
all_nav_msg_list_tmp = []

In [8]:
import pandas as pd
import numpy as np
import pymap3d as pm

gt_data = pd.read_csv("/persist/catkin_ws/src/GraphGNSSLib/global_fusion/dataset/gps_solution_TST/groundTruth_TST.csv", sep=",", header=None, names=["week", "tow", "lat", "lon", "alt"])
gt_data["timestamp"] = gt_data.apply(lambda x: GPSTime(x["week"], x["tow"]).as_datetime().timestamp(), axis=1)
gt_data = gt_data.drop(["week", "tow"], axis=1)
gt_ecef = gt_data.apply(lambda x: pm.geodetic2ecef(x["lat"], x["lon"], x["alt"]), axis=1)
gt_data["ecef_x"] = gt_ecef.apply(lambda x: x[0])
gt_data["ecef_y"] = gt_ecef.apply(lambda x: x[1])
gt_data["ecef_z"] = gt_ecef.apply(lambda x: x[2])
gt_data.to_csv("/persist/catkin_ws/src/GraphGNSSLib/global_fusion/dataset/gps_solution_TST/groundTruth_TST_ecef.csv", index=False)

In [9]:
from sensor_msgs.msg import NavSatFix
import rospy
gt_msgs = []
for i in range(gt_data.shape[0]):
    gt_msg = NavSatFix()
    gt_msg.header.stamp = rospy.Time.from_sec(gt_data.iloc[i]["timestamp"])
    gt_msg.header.frame_id = "ground_truth"
    gt_msg.header.seq = i
    gt_msg.latitude = gt_data.iloc[i]["lat"]
    gt_msg.longitude = gt_data.iloc[i]["lon"]
    gt_msg.altitude = gt_data.iloc[i]["alt"]
    gt_msgs.append((gt_data.iloc[i]["timestamp"], gt_msg, '/groundtruth'))

In [10]:
all_bag_msg_list = bag_msg_list + all_nav_msg_list + gt_msgs
all_bag_msg_list.sort(key=lambda x: x[0])

In [11]:
import rosbag
import rospy
import time
import subprocess
# try:
#     roscore = subprocess.Popen('roscore')
#     time.sleep(1)  # wait a bit to be sure the roscore is really launched
# except:
#     print("roscore is already running")
bag = rosbag.Bag('test.bag', 'w')
iono_msg = StampedFloat64Array()
iono_msg.data = nav_dat_n.attrs['ionospheric_corr_GPS']
iono_msg.header.stamp = rospy.Time.from_sec(all_bag_msg_list[0][0])
bag.write('/gnss/iono_gps', iono_msg, t=rospy.Time.from_sec(all_bag_msg_list[0][0]))
iono_msg.data = nav_dat_b.attrs['ionospheric_corr_BDS']
iono_msg.header.stamp = rospy.Time.from_sec(all_bag_msg_list[0][0])
bag.write('/gnss/iono_bds', iono_msg, t=rospy.Time.from_sec(all_bag_msg_list[0][0]))
for timestamp, bag_msg, topic_name in all_bag_msg_list:
    # print(timestamp, bag_msg, topic_name)
    bag.write(topic_name, bag_msg, t=rospy.Time.from_sec(timestamp))
bag.close()